In [ ]:
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import os
import joblib
import time

In [2]:
def load_data():
    data = []
    labels = []

    for i in range(4):
        directory_path = os.path.join(os.getcwd(), 'dataset', str(i))
        file_list = os.listdir(directory_path)
        
        for filename in file_list:
            image_path = os.path.join(directory_path, filename)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            binary_image = preprocess_image(image)
            features = extract_features(binary_image)
            
            data.append(features)
            labels.append(i)

    return np.array(data), np.array(labels)

In [3]:
def preprocess_image(image):
    resized_image = cv2.resize(image, (10, 10))
    _, binary_image = cv2.threshold(resized_image, 128, 255, cv2.THRESH_BINARY)
    return binary_image

def extract_features(image):
    features = image.flatten()
    return features

def train_classifier(data, labels):
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    classifier = SVC()

    classifier.fit(X_train, y_train)

    joblib.dump(classifier, 'svm_model.joblib')

    predictions = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Accuracy: {accuracy}")

    return classifier

In [4]:
def load_classifier(model_path='svm_model.joblib'):
    return joblib.load(model_path)

def process_image(image_path, classifier):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    processed_image = preprocess_image(image)
    features = extract_features(processed_image)
    number = classifier.predict([features])[0]
    return number

In [5]:
if not os.path.exists('svm_model.joblib'):
    start_time = time.time()
    data, labels = load_data()
    classifier = train_classifier(data, labels)
    end_time = time.time()
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
else:
    classifier = load_classifier()

image_path = 'guess/DIEGO.jpg'
result = process_image(image_path, classifier)

print(f"The recognized number is: {result}")

Accuracy: 0.9606921082775668
Training time: 533.42 seconds
The recognized number is: 1
